In [221]:
import json
from datetime import datetime, date
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import os

In [222]:
with open("C:/GLOBAL POWER RANKINGS/esports-data/tournaments.json", "r") as json_file:
                tournaments_json = json.load(json_file)
        
with open("C:/GLOBAL POWER RANKINGS/esports-data/leagues.json", "r") as json_file:
                leagues_json = json.load(json_file)



In [223]:
def get_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ").date()

def convert_time_to_seconds(time_str):
    time = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ").time()
    return time.hour * 3600 + time.minute * 60 + time.second

In [224]:
def get_league(tournament):
    for t in tournaments_json:
        if t["slug"] == tournament:
            league_id = t["leagueId"]
    for league in leagues_json:
        if league["id"] ==league_id:
            league_name = league["name"]
            if league["region"] == "INTERNATIONAL":
                return None

    return league_name

In [225]:
get_league("lec_spring_2022")

'LEC'

In [226]:
def get_game_info(match, tournament,stage):
    
    game_info = match[0]
    game_end = match[-1]
    game_id = int((game_end["gameName"]).split("|")[0])
    #print(game_id)
    for t in tournaments_json:
        if t["slug"] == tournament:
            for s in t["stages"]:
                if s["name"] == stage:
                    for section in s["sections"]:
                        for m in section["matches"]:
                            if int(m["id"]) == game_id:
                                #print("id found")
                                best_of = m["strategy"]["count"]
                                break  # exits the loop when the match is found
                    #exits the other sections of the loop if best_of exits in the local scope
                        if "best_of" in locals():
                            break  
                    if "best_of" in locals():
                        break  
                if "best_of" in locals():
                    break  
            if "best_of" in locals():
                break  
                        
    game = pd.DataFrame()
    
    start_time = convert_time_to_seconds(game_info["eventTime"])
    #print(start_time)
    end_time = convert_time_to_seconds(game_end["eventTime"])
    date = get_date(game_info["eventTime"])

    game["date"] = [date]
    game["start_time"] = [start_time]
    game["end_time"] = [end_time]
    game["duration"] = [game.iloc[0]["end_time"] - game.iloc[0]["start_time"]]

    patch = game_info["gameVersion"]
    game["patch"] =[patch]

    participants =game_info["participants"]
    roles = ["top","jng","mid","adc","sup","top","jng","mid","adc","sup"]
    
    for player,role in zip(participants,roles):
        summonerName =player["summonerName"].split(" ")
        team_tag = summonerName.pop(0)
        player_name = " ".join(summonerName)
        if player["teamID"]==100:
            side = "blue"
        else:
            side = "red"
        game[f"{side}_team"]= [team_tag]
        game[f"{side}_{role}"]= [player_name]
    game["best_of"] = best_of
    game["stage"]=[stage]
    if game_end["winningTeam"]== 100:
        game["result"]=[1]
    else:
        game["result"]=[0]
    for side in ["blue","red"]:
        team = game.iloc[0][f"{side}_team"]
        if team in league_names:
            #print(league_names[team])
            game[f"{side}_league"]=league_names[team]
        else:
            league = get_league(tournament)
            league_names[team]= league
            game[f"{side}_league"] = league
            print(f"added {team} to league_names")
    return(game)

In [227]:
def get_status_updates(match,game):
    start_time = game.iloc[0]["start_time"]
    #print(start_time)
    roles = ["top","jng","mid","adc","sup","top","jng","mid","adc","sup"]
    end_time = int(convert_time_to_seconds(match[-2]["eventTime"]) - start_time)
    for event in match:
        event_time = int(convert_time_to_seconds(event["eventTime"]) - start_time)
        for time in ["600", "840", end_time]: #can add more times, 300 sec for 5 mins game time
            if (event["eventType"] == "stats_update") and (event_time == int(time)):
                if time == end_time:
                    time = "end"
                participants = event["participants"]
                variables =["totalGold","level"] #, "shutdownValue"
                #other stats that have been taken out "WARD_KILLED", "WARD_PLACED", "TOTAL_DAMAGE_DEALT", "TOTAL_DAMAGE_TAKEN"
                stats= ["NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE", "CHAMPIONS_KILLED","NUM_DEATHS", "ASSISTS", "VISION_SCORE", "TOTAL_DAMAGE_DEALT_TO_CHAMPIONS"]
                pings = [ "BASIC_PINGS", "COMMAND_PINGS","DANGER_PINGS","GET_BACK_PINGS","RETREAT_PINGS", "ON_MY_WAY_PINGS", "ASSIST_ME_PINGS", "ENEMY_MISSING_PINGS", "PUSH_PINGS", "ALL_IN_PINGS", "HOLD_PINGS", "BAIT_PINGS", "VISION_CLEARED_PINGS", "ENEMY_VISION_PINGS", "NEED_VISION_PINGS"]
                team_stats = ["inhibKills","towerKills","baronKills","dragonKills"]
                minions = ["MINIONS_KILLED","NEUTRAL_MINIONS_KILLED","NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE","NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE"]
                
                red_pings=0
                blue_pings =0
                for player,role in zip(participants,roles):
                    if player["teamID"]==100:
                        side = "blue"
                    else:
                        side = "red"
                    for var in variables:
                        game[f"{side}_{role}_{var}_{time}"] = [player[var]]
            
                    stat_list = player["stats"]
                    cs =0
                    for stat in stat_list:
                        if stat["name"] in minions:
                            name = stat["name"]
                            cs += stat["value"]
                            
                        if stat["name"] in stats:
                            name = stat["name"]
                            game[f"{side}_{role}_{name}_{time}"] = stat["value"]
                            
                        elif stat["name"] in pings:
                            if side =="blue":
                                blue_pings += stat["value"]
                            else:
                                red_pings += stat["value"]
    
                            
                    game[f"{side}_{role}_cs_{time}"] = [cs]
                game[f"blue_pings_{time}"] = blue_pings
                game[f"red_pings_{time}"] = red_pings
                for stat in team_stats:
                    game[f"blue_{stat}_{time}"] =[event["teams"][0][stat]]
                    game[f"red_{stat}_{time}"] =[event["teams"][1][stat]]
    return(game)

In [228]:
def differences(df):
    roles = ["top","jng","mid","adc","sup"]
    times= ["600","840"]

    for side, opp_side in zip(["blue","red"],["red","blue"]):
        for role in roles:
            for time in times:
                for stat in ["cs", "totalGold"]:
                    df[f"{side}_{role}_{stat}_diff_{time}"] = df[f"{side}_{role}_{stat}_{time}"] - df[f"{opp_side}_{role}_{stat}_{time}"]
    return(df)

In [229]:
def statspermin(df):
    roles = ["top","jng","mid","adc","sup"]

    for side in ["blue","red"]:
        for role in roles:
            for stat in ["cs", "totalGold", "VISION_SCORE", "TOTAL_DAMAGE_DEALT_TO_CHAMPIONS"]:
                df[f"{side}_{role}_{stat}_per_min"] = df[f"{side}_{role}_{stat}_end"] /(df["duration"] / 60)
    return(df)

In [230]:
#dictionary for team regions
league_names = {}
df = pd.DataFrame()
directory = "C:/GLOBAL POWER RANKINGS/games/"
for folder in os.listdir(directory):
    tournament = folder
    for stage in os.listdir(f"{directory}{folder}"):
        print(f" processing {directory}{folder}/{stage}")
        for games in os.listdir(f"{directory}{folder}/{stage}"):
            with open(f"{directory}{folder}/{stage}/{games}", "r") as json_file:
                match = json.load(json_file)
                game = get_game_info(match,tournament,stage)
                #print(game)
                game = get_status_updates(match,game)
                df =pd.concat([df,game])
                
df.head()

 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2022/Playoffs
added RGE to league_names
added MSF to league_names
added XL to league_names
added VIT to league_names
added G2 to league_names
added FNC to league_names
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2022/Regular Season
added AST to league_names
added SK to league_names
added MAD to league_names
added BDS to league_names
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Groups
added KOI to league_names
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Playoffs
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Regular Season
added TH to league_names
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2022/Playoffs
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2022/Regular Season
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2023/Groups
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2023/Playoffs
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2023/Regular

,date,start_time,end_time,duration,patch,blue_team,blue_top,blue_jng,blue_mid,blue_adc,...,blue_pings_end,red_pings_end,blue_inhibKills_end,red_inhibKills_end,blue_towerKills_end,red_towerKills_end,blue_baronKills_end,red_baronKills_end,blue_dragonKills_end,red_dragonKills_end
0,2022-03-25,61956,64249,2293,12.5.428.2127,RGE,Odoamne,Malrang,Larssen,Comp,...,0,0,2,0,9,3,2,0,3,3
0,2022-03-25,71591,72965,1374,12.5.428.2127,RGE,Odoamne,Malrang,Larssen,Comp,...,0,0,1,0,9,1,0,0,1,0
0,2022-03-27,54491,56737,2246,12.5.428.2127,XL,Finn,Markoon,Nukeduck,Patrik,...,0,0,2,0,11,4,2,0,4,2
0,2022-03-27,58002,59880,1878,12.5.428.2127,VIT,Alphari,Selfmade,Perkz,Carzzy,...,0,0,1,0,8,5,1,0,4,0
0,2022-03-27,61141,63428,2287,12.5.428.2127,XL,Finn,Markoon,Nukeduck,Patrik,...,0,0,1,1,7,7,0,1,2,2


In [231]:
#df = df.sort_values("start_time")
df = df.sort_values(by =["date", "start_time"])
df.reset_index(drop=True, inplace=True)
df_og =df

In [253]:
df =df_og

In [233]:
print(league_names)

{'RGE': 'LEC', 'MSF': 'LEC', 'XL': 'LEC', 'VIT': 'LEC', 'G2': 'LEC', 'FNC': 'LEC', 'AST': 'LEC', 'SK': 'LEC', 'MAD': 'LEC', 'BDS': 'LEC', 'KOI': 'LEC', 'TH': 'LEC'}


In [234]:
df_og.to_csv("lec_spring_2023_test.csv",index=True)

In [254]:
df = differences(df)
df = statspermin(df)

In [255]:
blue_cols = ["date","start_time", "duration"] +[col for col in df.columns if "blue" in col]
red_cols = ["date","start_time", "duration"] +[col for col in df.columns if "red" in col]
team_cols =[col.replace("blue_","") for col in blue_cols]
#print(team_cols)
blue = df[blue_cols]
red = df[red_cols]

In [256]:
#Get the tags of all of the teams
teams = df["blue_team"].drop_duplicates()
#make rge = koi etc

In [257]:
#separating the data for individual teams to get rolling averages
team_data = pd.DataFrame()
for team in teams:
    for index,row in df.iterrows():
        if row["blue_team"] == team:
            blue_renamed = blue.rename(columns=dict(zip(blue_cols, team_cols)))
            team_data = pd.concat([team_data, blue_renamed.iloc[[index]]])
        elif row["red_team"] == team:
            red_renamed = red.rename(columns=dict(zip(red_cols, team_cols)))
            team_data = pd.concat([team_data, red_renamed.iloc[[index]]])

In [258]:
team_data.to_csv("lec_spring_2023_teamdata.csv",index=True)

In [259]:
grouped_games = team_data.groupby("team")

In [260]:
def rolling_avg(group,cols,new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed ='left').mean() #assigns avg from previous 3 games onto the 4th game
    group[new_cols]=rolling_stats
    group = group.dropna(subset = new_cols) #drops the first 3 games without rolling averages
    return group

In [261]:
#need to change what cols it uses for rolling avg(instead of just _end ones)
cols = [col for col in team_data.columns if "_end" in col or "_300" in col or "_840" in col or "per_min" in col] + ["duration"] 
new_cols = [f"{c}_rolling" for c in cols]

In [262]:
games_rolling = grouped_games.apply(lambda x:rolling_avg(x,cols,new_cols))
#games_rolling

In [263]:
games_rolling = games_rolling.droplevel('team')
games_rolling

,date,start_time,duration,team,top,jng,mid,adc,sup,league,...,mid_VISION_SCORE_per_min_rolling_rolling,mid_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling,adc_cs_per_min_rolling_rolling,adc_totalGold_per_min_rolling_rolling,adc_VISION_SCORE_per_min_rolling_rolling,adc_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling,sup_cs_per_min_rolling_rolling,sup_totalGold_per_min_rolling_rolling,sup_VISION_SCORE_per_min_rolling_rolling,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling
31,2022-01-29,60568,2154,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,LEC,...,1.181255,538.517807,9.984965,414.518569,1.011076,455.731383,1.085216,202.117247,2.752737,100.851006
35,2022-02-04,61081,1864,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,LEC,...,1.267744,586.735776,9.544440,402.640847,0.990577,578.553959,1.016405,210.400595,2.897821,139.745854
42,2022-02-05,64848,2175,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,LEC,...,1.174804,509.928984,9.299187,397.157828,1.008087,601.307506,0.875445,216.749415,3.012081,162.712944
46,2022-02-11,64894,2211,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,LEC,...,1.160770,398.020652,9.423235,384.821226,0.987835,492.861590,0.931307,213.526613,3.107986,160.304454
53,2022-02-12,68407,1912,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,LEC,...,1.195642,345.695771,9.618861,385.718127,0.991113,348.386777,0.971360,215.046130,3.131697,146.587404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2023-07-29,61120,1822,XL,Odoamne,Peach,Abbedagge,Patrik,LIMIT,LEC,...,1.489187,814.694496,9.817888,469.633479,1.337468,788.061273,0.968616,245.296296,3.141840,180.664349
489,2023-07-29,67816,1702,XL,Odoamne,Peach,Abbedagge,Patrik,LIMIT,LEC,...,1.553965,860.072983,9.806017,472.817306,1.316360,796.329931,0.966416,241.710265,3.235828,175.220502
491,2023-07-30,61989,2234,XL,Odoamne,Peach,Abbedagge,Patrik,LIMIT,LEC,...,1.482174,897.113008,9.547575,448.517474,1.322137,662.611647,0.982388,235.309808,3.239785,165.077002
490,2023-07-30,58780,1938,XL,Odoamne,Peach,Abbedagge,Patrik,LIMIT,LEC,...,1.433335,902.532351,9.339228,449.978926,1.281270,602.729867,1.055183,246.651677,3.085007,162.293074


In [264]:
games = games_rolling.sort_index()
games.head()

,date,start_time,duration,team,top,jng,mid,adc,sup,league,...,mid_VISION_SCORE_per_min_rolling_rolling,mid_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling,adc_cs_per_min_rolling_rolling,adc_totalGold_per_min_rolling_rolling,adc_VISION_SCORE_per_min_rolling_rolling,adc_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling,sup_cs_per_min_rolling_rolling,sup_totalGold_per_min_rolling_rolling,sup_VISION_SCORE_per_min_rolling_rolling,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling
30,2022-01-29,57379,1765,SK,Jenax,Gilius,Sertuss,Jezu,Treatz,LEC,...,1.231079,489.695758,9.451327,375.802668,1.127169,497.458531,0.807812,204.709177,2.543714,162.134865
31,2022-01-29,60568,2154,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,LEC,...,1.181255,538.517807,9.984965,414.518569,1.011076,455.731383,1.085216,202.117247,2.752737,100.851006
31,2022-01-29,60568,2154,XL,Finn,Markoon,Nukeduck,Patrik,Mikyx,LEC,...,1.026223,531.644922,9.805800,436.789295,1.187123,612.556768,0.979366,215.293577,3.015808,90.980661
32,2022-01-29,64106,1703,MSF,HiRit,Shlatan,Vetheo,Neon,Mersa,LEC,...,1.158602,448.434049,9.868293,430.847030,1.114924,501.247210,0.745304,217.222243,2.336508,119.992886
32,2022-01-29,64106,1703,MAD,Armut,Elyoya,Reeker,UNF0RGIVEN,Kaiser,LEC,...,0.890458,434.940737,10.598154,486.502718,1.117661,616.050010,0.975856,225.264851,2.698033,99.990827


In [246]:
games.to_csv("games.csv", index = True)

In [205]:
#games.head(455)

In [265]:
# Recombining separate game data into the main set
rolling_cols = [col for col in games.columns if "rolling" in col]
blue_rolling_cols = [f"blue_{col}" for col in rolling_cols]
red_rolling_cols = [f"red_{col}" for col in rolling_cols]
#print(red_rolling_cols)
result_df = pd.DataFrame()
for team in teams:
    for index, row in df.iterrows():
        try:
            if index in games.index and row["blue_team"] == team:
                # Copy the original game DataFrame and rename columns
                game = games.rename(columns=dict(zip(rolling_cols, blue_rolling_cols)))
                # Adding in the rolling columns
                game = game[(game.index == index) & (game["team"] == team)]
                #print(game.index)
                new_cols = game.loc[index,blue_rolling_cols]
                df.loc[index, blue_rolling_cols]= new_cols
            elif index in games.index and row["red_team"] == team:
                # Copy the original game DataFrame and rename columns
                game = games.rename(columns=dict(zip(rolling_cols, red_rolling_cols)))
                game = game[(game.index == index) & (game["team"] == team)]
                game
                new_cols = game.loc[index,red_rolling_cols]
                df.loc[index, red_rolling_cols]= new_cols
        except KeyError:
            #its skipping the first 3 games for "new" teams as this data has been removed
            print(f"{index} for {team} not found")
            continue

384 for MAD not found
385 for MAD not found
386 for MAD not found
389 for MAD not found
390 for MAD not found
397 for MAD not found
452 for MAD not found
456 for MAD not found
457 for MAD not found
470 for MAD not found
471 for MAD not found
33 for RGE not found
36 for RGE not found
44 for RGE not found
46 for RGE not found
52 for RGE not found
237 for RGE not found
238 for RGE not found
239 for RGE not found
251 for XL not found
259 for XL not found
264 for XL not found
55 for AST not found
60 for AST not found
71 for AST not found
76 for AST not found
80 for AST not found
184 for FNC not found
191 for FNC not found
195 for FNC not found
200 for FNC not found
206 for FNC not found
208 for FNC not found
244 for FNC not found
256 for FNC not found
455 for SK not found
458 for SK not found
459 for SK not found
465 for SK not found
466 for SK not found
467 for SK not found
58 for G2 not found
64 for G2 not found
69 for G2 not found
78 for G2 not found
252 for G2 not found
258 for G2 not f

In [266]:
df = df.dropna()
df.head()

,date,start_time,end_time,duration,patch,blue_team,blue_top,blue_jng,blue_mid,blue_adc,...,red_mid_VISION_SCORE_per_min_rolling_rolling,red_mid_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling,red_adc_cs_per_min_rolling_rolling,red_adc_totalGold_per_min_rolling_rolling,red_adc_VISION_SCORE_per_min_rolling_rolling,red_adc_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling,red_sup_cs_per_min_rolling_rolling,red_sup_totalGold_per_min_rolling_rolling,red_sup_VISION_SCORE_per_min_rolling_rolling,red_sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling
31,2022-01-29,60568,62722,2154,12.2.419.1399,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,...,1.026223,531.644922,9.805800,436.789295,1.187123,612.556768,0.979366,215.293577,3.015808,90.980661
32,2022-01-29,64106,65809,1703,12.2.419.1399,MAD,Armut,Elyoya,Reeker,UNF0RGIVEN,...,1.158602,448.434049,9.868293,430.847030,1.114924,501.247210,0.745304,217.222243,2.336508,119.992886
34,2022-01-29,70702,72975,2273,12.2.419.1399,FNC,Wunder,Razork,Humanoid,Upset,...,1.024962,330.028360,9.321431,413.445206,1.101878,429.663259,1.209846,231.409191,3.202088,147.708027
35,2022-02-04,61081,62945,1864,12.2.419.1399,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,...,1.347691,454.125134,9.578413,368.827161,1.054293,487.378515,0.879127,200.380683,2.510147,138.072310
37,2022-02-04,67108,68833,1725,12.2.419.1399,VIT,Alphari,Selfmade,Perkz,Carzzy,...,1.117835,518.827104,9.881884,435.497964,1.075522,555.487837,0.815184,227.738809,2.362207,123.782097


In [81]:
#Converting categorical variables into numerical
#roles = ["top","jng","mid","adc","sup"]
#for side in ["blue","red"]:
    #for role in roles:
        #df[f"{side}_{role}_code"] =  df[f"{side}_{role}"].astype("category").cat.codes
    #df[f"{side}_team_code"] = df[f"{side}_team"].astype("category").cat.codes

In [269]:
roles = ["top","jng","mid","adc","sup"]
code_cols = []
for side in ["blue","red"]:
    for role in roles:
        code_cols += [col for col in df.columns if f"{side}_{role}" == col]
    code_cols += [f"{side}_team", f"{side}_league"]
code_cols += ["best_of"]
new_code_cols =[name + '_code' for name in code_cols]
df.rename(columns = dict(zip(code_cols,new_code_cols)), inplace=True)

In [271]:
df.info()
df.best_of_code = df.best_of_code.astype(str)
dates= df["date"]
df= df.drop(columns=["date", "patch"])
df_dummies = pd.get_dummies(df)
df_dummies["date"] = dates
df["date"] = dates

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 31 to 492
Columns: 924 entries, date to red_sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_per_min_rolling_rolling
dtypes: float64(842), int64(65), object(17)
memory usage: 2.4+ MB


In [272]:
df_dummies.to_csv("data_one_hot.csv",index=False)

In [274]:
code_cols = [col for col in df_dummies.columns if "_code" in col]
rolling_cols =[col for col in df_dummies.columns if "rolling" in col]
features = ["start_time"] + code_cols + rolling_cols
print(features)

['start_time', 'blue_team_code_AST', 'blue_team_code_BDS', 'blue_team_code_FNC', 'blue_team_code_G2', 'blue_team_code_KOI', 'blue_team_code_MAD', 'blue_team_code_MSF', 'blue_team_code_RGE', 'blue_team_code_SK', 'blue_team_code_TH', 'blue_team_code_VIT', 'blue_team_code_XL', 'blue_top_code_Adam', 'blue_top_code_Agresivoo', 'blue_top_code_Alphari', 'blue_top_code_Armut', 'blue_top_code_BrokenBlade', 'blue_top_code_Chasy', 'blue_top_code_Evi', 'blue_top_code_Finn', 'blue_top_code_HiRit', 'blue_top_code_Irrelevant', 'blue_top_code_JNX', 'blue_top_code_Jenax', 'blue_top_code_Odoamne', 'blue_top_code_Oscarinin', 'blue_top_code_Photon', 'blue_top_code_Szygenda', 'blue_top_code_Vizicsacsi', 'blue_top_code_WhiteKnight', 'blue_top_code_Wunder', 'blue_jng_code_113', 'blue_jng_code_Bo', 'blue_jng_code_Cinkrof', 'blue_jng_code_Daglas', 'blue_jng_code_Elyoya', 'blue_jng_code_Gilius', 'blue_jng_code_Haru', 'blue_jng_code_Jankos', 'blue_jng_code_Malrang', 'blue_jng_code_Markoon', 'blue_jng_code_Peach'

In [52]:
X = games[features]
y = games["result"]

In [53]:
X

,side_code,start_time,team_code,top_code,jng_code,mid_code,adc_code,sup_code,enemy_top_code,enemy_jng_code,...,sup_VISION_SCORE_end_rolling,sup_TOTAL_DAMAGE_DEALT_end_rolling,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling,sup_TOTAL_DAMAGE_TAKEN_end_rolling,inhibKills_end_rolling,towerKills_end_rolling,baronKills_end_rolling,dragonKills_end_rolling,total_pings_rolling,duration_rolling
0,0,61709,0,4,0,3,6,3,2,2,...,99.860382,16794.774740,4427.193197,12957.264160,0.666667,5.666667,0.666667,1.666667,630.333333,1728.000000
0,1,61709,5,2,2,5,0,2,4,0,...,72.915736,17991.160807,5415.185791,15793.098958,0.333333,4.666667,0.333333,2.000000,439.666667,1821.333333
0,1,75931,3,1,9,1,4,8,9,4,...,89.894414,25134.458333,9303.239746,11583.531250,1.000000,7.333333,0.666667,3.333333,387.000000,1658.333333
0,1,72355,6,5,5,8,3,1,8,1,...,94.017652,17294.555990,4520.162760,9673.050456,1.000000,8.333333,1.000000,3.000000,521.333333,1881.666667
0,0,68295,2,7,6,2,8,0,0,7,...,81.961739,16381.268880,7043.188477,7767.103516,0.000000,2.000000,0.000000,1.000000,240.333333,1767.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,65139,1,0,7,10,2,6,2,2,...,82.417299,18332.043294,4220.047526,13524.517904,1.000000,5.666667,0.666667,2.666667,578.666667,1972.666667
0,0,70823,1,0,7,10,2,6,2,2,...,101.628176,19194.934245,4268.654785,11593.568034,1.333333,7.666667,1.666667,3.666667,662.333333,2187.666667
0,1,61837,1,0,7,10,2,6,2,2,...,88.095445,17717.075195,3267.629883,10769.328125,1.000000,6.666667,1.000000,2.666667,580.000000,1970.333333
0,0,61837,5,2,2,5,0,2,0,7,...,76.946331,18592.972331,2829.449300,12599.928060,2.333333,7.000000,1.000000,2.333333,521.000000,1784.666667
